In [ ]:
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import igraph as ig
import os
from scipy.stats import pearsonr

In [ ]:
import BraiAn

path_to_allen_json = "./data/AllenMouseBrainOntology.json"
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json)
colours = AllenBrain.get_region_colours()

In [ ]:
experiment = "soumnya"
group_1_animal_names = "C"
group_2_animal_names = "S"
marker = "CFos"
normalization = "Density"
comparison_folder = "CM-SM"

filename = f"{marker.lower()}_{normalization.lower()}.csv"
filename_path = os.path.join(".", "data", "experiments", experiment, "BraiAn_PLS_output", comparison_folder, filename)

In [ ]:
experiment = "soumnya"
group_1_animal_names = "C"
group_2_animal_names = "S"
marker = "CFos"
normalization = "RelativeDensity"
comparison_folder = "CF-SF"

filename = f"{marker.lower()}_{normalization.lower()}.csv"
filename_path = os.path.join(".", "data", "experiments", experiment, "BraiAn_PLS_output", comparison_folder, filename)

In [ ]:
group_1 = pd.read_csv(filename_path, sep=";", usecols=lambda c: group_1_animal_names in c or c == "BrainRegion")
assert "BrainRegion" in group_1.columns, f"'{filename}' is expected to have a column called 'BrainRegion'."
group_1 = group_1.set_index("BrainRegion")
group_1.reindex(index=group_1.index[::-1])
group_1_r = group_1.transpose().corr(method=lambda x,y: pearsonr(x,y)[0], min_periods=4)
group_1_p = group_1.transpose().corr(method=lambda x,y: pearsonr(x,y)[1], min_periods=4)

In [ ]:
group_2 = pd.read_csv(filename_path, sep=";", usecols=lambda c: group_2_animal_names in c or c == "BrainRegion")
assert "BrainRegion" in group_2.columns, f"'{filename}' is expected to have a column called 'BrainRegion'."
group_2 = group_2.set_index("BrainRegion")
group_2.reindex(index=group_2.index[::-1])
group_2_r = group_2.transpose().corr(method=lambda x,y: pearsonr(x,y)[0], min_periods=4)
group_2_p = group_2.transpose().corr(method=lambda x,y: pearsonr(x,y)[1], min_periods=4)

In [ ]:
p_cutoff = 0.05
r_cutoff = 0.9

group_2_valid_edges = (group_2_p.abs() <= p_cutoff) & (group_2_r.abs() >= r_cutoff)
M = group_2_r.copy(deep=True)
M[~group_2_valid_edges] = 0

# L=9
# group_2_valid_edges = group_2_valid_edges.iloc[0:L, 0:L]
# M = group_2_r.iloc[0:L, 0:L][group_2_valid_edges]
# M[~group_2_valid_edges] = 0

In [ ]:
regions_MD = AllenBrain.get_areas_major_division(*M.index)
# sort vertices based on major divisions
regions_i = sorted(range(len(M)), key=lambda i: BraiAn.MAJOR_DIVISIONS.index(list(regions_MD.values())[i])) # np.array(list(regions_MD.values())).argsort()

active_MD = sorted(list(set(regions_MD.values())), key=BraiAn.MAJOR_DIVISIONS.index) # np.sort(np.unique(np.array(list(regions_MD.values()))))
n_MD = [sum(r1 == r2  for r2 in regions_MD.values()) for r1 in active_MD]

In [ ]:
# G = ig.Graph.Read_GML('Eurovision15.gml')
# G = ig.Graph.Weighted_Adjacency(G.get_adjacency()._data)
M = M.iloc[regions_i].iloc[:,regions_i]
G = ig.Graph.Weighted_Adjacency(M.values, mode="lower") # mode="undirected")
G.vs["label"] = list(M.index)

V = np.array(G.vs)
region_labels = [f"Region: <b>{v['label']}</b><br>Major Division: {regions_MD[v['label']]}" for v in V]

In [ ]:
E=[e.tuple for e in G.es]# list of edges
len(E)

In [ ]:
layt=G.layout_circle() # layout('circular') #circular layout

In [ ]:
L=len(layt)
layt[7]

In [ ]:
r_values = G.es["weight"] # list(map(int, G.es["weight"]))

In [ ]:
def dist(A,B):
    return np.linalg.norm(np.array(A)-np.array(B))
dist(layt[0], layt[5])

In [ ]:
Dist=[0, dist([1,0], 2*[np.sqrt(2)/2]), np.sqrt(2), dist([1,0],  [-np.sqrt(2)/2, np.sqrt(2)/2]), 2.0]
params=[1.2, 1.5, 1.8, 2.1]

In [ ]:
def get_idx_interv(d, D):
    k=0
    while(d>D[k]):
        k+=1
    return  k-1

In [ ]:
class InvalidInputError(Exception):
    pass

def deCasteljau(b,t):
    N=len(b)
    if(N<2):
        raise InvalidInputError("The  control polygon must have at least two points")
    a=np.copy(b) #shallow copy of the list of control points 
    for r in range(1,N):
        a[:N-r,:]=(1-t)*a[:N-r,:]+t*a[1:N-r+1,:]
    return a[0,:]

def BezierCv(b, nr=5):
    t=np.linspace(0, 1, nr)
    return np.array([deCasteljau(b, t[k]) for k in range(nr)])

In [ ]:
node_color = [colours[v["label"]] if v.degree() > 0 else '#CCCCCC' for v in G.vs] # ['rgba(0,51,181, 0.85)'  if v['label'] in Contestant else '#CCCCCC' for v in G.vs]
line_color = ['#FFFFFF' if v.degree() > 0 else 'rgb(150,150,150)' for v in G.vs]
# edge_colors = ["red", "orange", "green", "blue"]
edge_colors = ['#d4daff','#84a9dd', '#5588c8', '#6d8acf']

In [ ]:
Xn=[layt[k][0] for k in range(L)]
Yn=[layt[k][1] for k in range(L)]

In [ ]:
def get_edges_widths(r_values, r_cutoff, max=5):
    return (np.abs(np.array(r_values))-r_cutoff)/(1-r_cutoff)*max
edges_widths = get_edges_widths(r_values, r_cutoff)

In [ ]:
lines = [] # the list of dicts defining   edge  Plotly attributes
edge_info = [] # the list of points on edges where  the information is placed

for j, e in enumerate(E):
    A=np.array(layt[e[0]])
    B=np.array(layt[e[1]])
    d=dist(A, B)
    K=get_idx_interv(d, Dist)
    b=[A, A/params[K], B/params[K], B]
    color=edge_colors[K]
    pts=BezierCv(b, nr=5)
    text=f"<b>{V[e[0]]['label']} - {V[e[1]]['label']}</b><br>r: {r_values[j]}"
    mark=deCasteljau(b,0.9)
    edge_info.append(go.Scatter(x=[mark[0]],
                             y=[mark[1]],
                             mode='markers',
                             marker=dict( size=0.5,  color=edge_colors),
                             text=text,
                             hoverinfo='text'
                             )
                    )
    lines.append(go.Scatter(x=pts[:,0],
                         y=pts[:,1],
                         mode='lines',
                         line=dict(color=color,
                                  shape='spline',
                                  width=edges_widths[j] #Weights[j]/5#The  width is proportional to the edge weight
                                 ),
                        hoverinfo='none'
                       )
                )

In [ ]:
trace2=go.Scatter(x=Xn,
           y=Yn,
           mode='markers',
           name='',
           marker=dict(symbol='circle',
                         size=15,
                         color=node_color,
                         line=dict(color=line_color, width=0.5)
                         ),
           text=region_labels,
           hoverinfo='text',
           )

axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

In [ ]:
def make_annotation(anno_text, y_coord):
    return dict(showarrow=False,
                      text=anno_text,
                      xref='paper',
                      yref='paper',
                      x=0,
                      y=y_coord,
                      xanchor='left',
                      yanchor='bottom',
                      font=dict(size=12)
                     )

In [ ]:
anno_text1='Blue nodes mark the countries that are both contestants and jury members'
anno_text2='Grey nodes mark the countries that are only jury members'
anno_text3='There is an edge from a Jury country to a contestant country '+\
           'if the jury country assigned at least one vote to that contestant'
width=1500
height=1500
title="A connectomics graph of Pearson correlation data"
layout=go.Layout(
              title= title,
              font= dict(size=12),
              showlegend=False,
              autosize=False,
              width=width,
              height=height,
              xaxis=dict(axis),
              yaxis=dict(axis,
                        scaleanchor="x", scaleratio=1),
              margin=dict(l=40,
                            r=40,
                            b=85,
                            t=100,
                          ),
              hovermode='closest',
              plot_bgcolor='rgba(0,0,0,0)'
#              annotations=list([make_annotation(anno_text1, -0.07),
#                                       make_annotation(anno_text2, -0.09),
#                                       make_annotation(anno_text3, -0.11)]
#                                     )
              )

In [ ]:
PI=np.pi

def moduloAB(x, a, b): #maps a real number onto the unit circle identified with 
                       #the interval [a,b), b-a=2*PI
        if a>=b:
            raise ValueError('Incorrect interval ends')
        y=(x-a)%(b-a)
#        print(f"moduloAB({x}, {a}, {b}): {y+b if y<0 else y+a}")
        return y+b if y<0 else y+a

def test_2PI(x):
    return 0<= x <2*PI

ideogram_length=2*PI*np.asarray(n_MD)/sum(n_MD)

def get_ideogram_ends(ideogram_len):
    ideo_ends=[]
    left=0#-2*PI/(2*sum(n_MD)) # 0
    for k in range(len(ideogram_len)):
        right=left+ideogram_len[k]
        ideo_ends.append([left, right])
        left=right
    return ideo_ends

ideo_ends = get_ideogram_ends(ideogram_length)
ideo_ends = np.array(ideo_ends)-PI/sum(n_MD)


def make_ideogram_arc(R, phi, a=50):
    # R is the circle radius
    # phi is the list of ends angle coordinates of an arc
    # a is a parameter that controls the number of points to be evaluated on an arc
#    if not test_2PI(phi[0]) or not test_2PI(phi[1]):
#        phi=[moduloAB(t, 0, 2*PI) for t in phi]
    length=(phi[1]-phi[0])% 2*PI
    nr=5 if length<=PI/4 else int(a*length/PI)

    if phi[0] < phi[1]:
        theta=np.linspace(phi[0], phi[1], nr)
    else:
        phi=[moduloAB(t, -PI, PI) for t in phi]
        theta=np.linspace(phi[0], phi[1], nr)
    return R*np.exp(1j*theta)


In [ ]:
ideo_colors = [colours[r_acronym] for r_acronym in active_MD]

In [ ]:
def make_ideo_shape(path, line_color, fill_color):
    #line_color is the color of the shape boundary
    #fill_collor is the color assigned to an ideogram
    return  dict(
                  line=dict(
                  color=line_color,
                  width=0.45
                 ),

            path=  path,
            type='path',
            fillcolor=fill_color,
            layer='below'
        )

In [ ]:
ideograms=[]
for k in range(len(ideo_ends)):
    z= make_ideogram_arc(1.1, ideo_ends[k])
    zi=make_ideogram_arc(1.0, ideo_ends[k])
    m=len(z)
    n=len(zi)
    ideograms.append(go.Scatter(x=z.real,
                             y=z.imag,
                             mode='lines',
                             line=dict(color=ideo_colors[k], shape='spline', width=0.25),
                             text=active_MD[k]+'<br>'+'{:d}'.format(n_MD[k]),
                             hoverinfo='text'
                             )
                     )


    path='M '
    for s in range(m):
        path+=str(z.real[s])+', '+str(z.imag[s])+' L '

    Zi=np.array(zi.tolist()[::-1])

    for s in range(m):
        path+=str(Zi.real[s])+', '+str(Zi.imag[s])+' L '
    path+=str(z.real[0])+' ,'+str(z.imag[0])

    layout.shapes = [*layout.shapes, make_ideo_shape(path,'rgb(150,150,150)' , ideo_colors[k])]

In [ ]:
data=lines+edge_info+[trace2]
fig=go.Figure(data=data+ideograms, layout=layout)
fig.show()

In [ ]:
fig.write_image("stress_M_chord_diagram.svg")